# Project 2: Market-basket analysis - IMDB dataset

Project for the course of Algorithms for Massive Dataset <br> Nicolas Facchinetti 961648 <br> Antonio Belotti 960822

# Load preprocessed dataset from file data.zip

Use the code below do load the dataset from a preprocessed file data.zip

In [1]:
from google.colab import files

uploaded = files.upload()
  
!unzip data.zip && rm data.zip

Saving data.zip to data.zip
Archive:  data.zip
   creating: data/
  inflating: data/part-00063-717f407c-435f-4def-bca7-0ae425d828a4-c000.json  
 extracting: data/.part-00174-717f407c-435f-4def-bca7-0ae425d828a4-c000.json.crc  
 extracting: data/.part-00180-717f407c-435f-4def-bca7-0ae425d828a4-c000.json.crc  
  inflating: data/part-00071-717f407c-435f-4def-bca7-0ae425d828a4-c000.json  
  inflating: data/part-00187-717f407c-435f-4def-bca7-0ae425d828a4-c000.json  
 extracting: data/.part-00017-717f407c-435f-4def-bca7-0ae425d828a4-c000.json.crc  
 extracting: data/.part-00062-717f407c-435f-4def-bca7-0ae425d828a4-c000.json.crc  
 extracting: data/.part-00140-717f407c-435f-4def-bca7-0ae425d828a4-c000.json.crc  
 extracting: data/.part-00114-717f407c-435f-4def-bca7-0ae425d828a4-c000.json.crc  
  inflating: data/part-00003-717f407c-435f-4def-bca7-0ae425d828a4-c000.json  
 extracting: data/.part-00091-717f407c-435f-4def-bca7-0ae425d828a4-c000.json.crc  
  inflating: data/part-00138-717f407c-435

In [6]:
data = spark.read.format("json").option("header", "true").load("data").select('tconst', 'nconsts').rdd

In [ ]:
data.take(5)

[Row(tconst='tt0000009', nconsts=['nm0063086', 'nm0183823', 'nm1309758']),
 Row(tconst='tt0000335', nconsts=['nm1010955', 'nm1012612', 'nm1011210', 'nm1012621', 'nm0675239', 'nm0675260']),
 Row(tconst='tt0000502', nconsts=['nm0215752', 'nm0252720']),
 Row(tconst='tt0000574', nconsts=['nm0846887', 'nm0846894', 'nm3002376', 'nm0170118']),
 Row(tconst='tt0000615', nconsts=['nm3071427', 'nm0581353', 'nm0888988', 'nm0240418', 'nm0346387', 'nm0218953'])]

# Set up the Spark enviorment

We start by dowloading and installing all the needed tool to deal with Spark. In particular we are interested in obtainig a Java enviorment since Spark in written in Scala and so it need a JVM to run. Then we can download Apache Spark 3.1.2 with Hadoop 3.2 by the Apache CDN and uncompress it. Finally we can get and install PySpark, an interface for Apache Spark in Python

In [3]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget https://dlcdn.apache.org/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz
!tar xf spark-3.1.2-bin-hadoop3.2.tgz
!pip install -q findspark

--2021-09-08 07:50:13--  https://dlcdn.apache.org/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz
Resolving dlcdn.apache.org (dlcdn.apache.org)... 151.101.2.132, 2a04:4e42::644
Connecting to dlcdn.apache.org (dlcdn.apache.org)|151.101.2.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 228834641 (218M) [application/x-gzip]
Saving to: ‘spark-3.1.2-bin-hadoop3.2.tgz’

spark-3.1.2-bin-had 100%[===================>] 218.23M   192MB/s    in 1.1s    

2021-09-08 07:50:15 (192 MB/s) - ‘spark-3.1.2-bin-hadoop3.2.tgz’ saved [228834641/228834641]



The next step is to correctly set the path in our remote enviorment to use the obtained tools.

In [4]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop3.2"

Finally we can import PySpark in the project

In [5]:
import findspark
findspark.init("spark-3.1.2-bin-hadoop3.2")# SPARK_HOME
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

# Download the dataset from Kaggle

First install the Python module of Kaggle to download the dataset from its datacenter

In [ ]:
!pip install kaggle

Then load kaggle.json, a file containing your API credentials to be able to use the services offered by Kaggle

In [ ]:
from google.colab import files

uploaded = files.upload()
  
# Move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

Now we can download the dataset

In [ ]:
!kaggle datasets download 'ashirwadsangwan/imdb-dataset'

We now must unzip the compressed archive to use it. Once done we can also remove it.

In [ ]:
!unzip imdb-dataset.zip && rm imdb-dataset.zip

# Preapare the data for Spark

We can directly load the downloaded and extracted .tsv file in a Spark DataFrame by using the command read.csv(). We directly pass to the method the columns in which we are interested.

In [ ]:
df_principals = spark.read.csv("/content/title.principals.tsv/title.principals.tsv", sep=r'\t', header=True).select('tconst','nconst','category')

In [ ]:
df_principals.show(10)

In [ ]:
df_basics = spark.read.csv("/content/title.basics.tsv/title.basics.tsv", sep=r'\t', header=True).select('tconst','titleType')

In [ ]:
df_basics.show(10)

By inspecting the content of the column 'category' of df_principlas we can see that there are many jobs other than actors and actress (which are the two we are interested in)

In [ ]:
df_principals.select("category").distinct().show()

Similarly we can do the same thing with df_basics and the column 'titleType' to see how many categories a title can have.

In [ ]:
df_basics.select("titleType").distinct().show()

Once the data is loaded in a Spark DataFrame we can use the PySpark SQL module for processing the data. We start by exctracting only actors and actress from df_principals

In [ ]:
pre = df_principals.count()
df_principals.createOrReplaceTempView("PRINCIPALS") # create a temporary table on DataFrame
df_principals = spark.sql("SELECT * from PRINCIPALS WHERE category ='actor' OR category='actress'")
print("We reduced the number of row from {} to {}".format(pre, df_principals.count()))

 And then we do the same thing with movies in df_basics

In [ ]:
pre = df_basics.count()
df_basics.createOrReplaceTempView("BASICS") # create a temporary table on DataFrame
df_basics = spark.sql("SELECT * from BASICS WHERE titleType ='movie'")
print("We reduced the number of row from {} to {}".format(pre, df_basics.count()))

We can now see that we have two DataFrame, one containing only the movies and the other only the people which play as actor/actress in a title. To do the desired maket-basket analysis we have to pivot our tconst as rows, so each row stands for one titleId, and then including a list of nconst identifiers of the actors that played in it.

In [ ]:
df_basics.show(10)

In [ ]:
df_principals.show(10)

So we start by joining the two dataframe to extract from df_principals only the records with tconst related to a movie. We can also discard the category column since is no longer usefull.

In [ ]:
basket_data = df_principals.join(df_basics, "tconst").select(df_principals.tconst, df_principals.nconst).sort("tconst")

In [ ]:
basket_data.show(10)

Then we can remove hypothetical duplicated row and then aggregate the data using tconst identifier.

In [ ]:
from pyspark.sql import functions as F
basket_data = basket_data.dropDuplicates()
basket_data = basket_data.groupBy("tconst").agg(F.collect_list("nconst").alias("nconsts")).sort('tconst')

In [ ]:
print("There are {} titleId buckets".format(basket_data.count()))
basket_data.show(10, False)

As we can see above we now have the data in the correct format to do our analysis: in each row we have the identifier of a movie and in the second column the list of the idenfiers of the actors that played in it.
Since we done all the needed pre-processing computation on the data we can transform our DataFrame in a RDD to apply map-reduce functions.

Serialize to file the RDD and download to skip the processing all the time.



In [ ]:
basket_data.write.format('json').save("data")

In [ ]:
!zip -r data.zip data

In [ ]:
from google.colab import files
files.download('data.zip')

# Prove Map-reduce

Accediamo al campo 1 sicchè 0 è il bucket, flat perché cosi unisce tutte le row in una

In [ ]:
data.flatMap(lambda row: row[1]).take(10)

Mappiamo ogni record di autore trovato in se stesso e 1

In [ ]:
data.flatMap(lambda row: (row[1]))
  .map(lambda elem: (elem,1)).take(10)

Aggiungiamo reduce che somma la parte dopo il contantore dell'attore

In [ ]:
data.flatMap(lambda row: (row[1])).map(lambda elem: (elem,1)).reduceByKey(lambda a,b: a+b).take(10)

Aggiungiamo un threshold (almeno 200 apparizioni)

In [ ]:
res = data.flatMap(lambda row: (row[1])) \
          .map(lambda elem: (elem,1)) \
          .reduceByKey(lambda a,b: a+b) \
          .filter(lambda x: x[1] >=200)
res.take(10)

Vediamo ora per la seconda parte di apriori

In [ ]:
data.take(10)

Prendiamo il primo record per provare e estriamo i due elementi. Scriviamo una funzione che controlla se gli elementi di una copia sono nella riga

In [ ]:
coppia = ['nm0063086', 'nm0183823']    #primi due attori del primo record

def row_contains_elements(row, elements):
  return all(x in row for x in elements)

data.map(lambda x:x[1]).filter(lambda x: row_contains_elements(x,coppia)).take(5)


Proviamo ora a cercare di far generare le copie possibili ad ogni singola riga. trick per evitare doppioni. flatmap direttamente almeno sono gia spacchettate

In [ ]:
data.take(1)

In [ ]:
def generate_candidate(x):
  candidates = []
  for a, elemA in enumerate(x):
    for b, elemB in enumerate(x):
      if a < b:
        candidates.append((elemA, elemB))
  return candidates

data.map(lambda x: x[1]).flatMap(lambda x: generate_candidate(x)).take(10)

Aggiungiamo poi un controllo che la copia generata sia in quelle di interesse

In [ ]:
copia = [('nm0063086', 'nm0183823'), ('nm0846894', 'nm3002376')]

def generate_candidate(x):
  candidates = []
  for a, elemA in enumerate(x):
    for b, elemB in enumerate(x):
      if a < b:
        candidates.append((elemA, elemB))
  return candidates

data.map(lambda x: x[1]).flatMap(lambda x: generate_candidate(x)).filter(lambda x: x in copia).take(3)

Vero proprio passo di map. Le tuple per qualche motivo sono hashabili

In [ ]:
copia = [('nm0063086', 'nm0183823'), ('nm0846894', 'nm3002376')]

def generate_candidate(x):
  candidates = []
  for a, elemA in enumerate(x):
    for b, elemB in enumerate(x):
      if a < b:
        candidates.append((elemA, elemB))
  return candidates

data.map(lambda x: x[1]).flatMap(lambda x: generate_candidate(x)) \
    .filter(lambda x: x in copia).map(lambda x: (x,1)).take(3)

Aggiungiamo reduce e il controllo del threshold

In [ ]:
copia = [('nm0063086', 'nm0183823'), ('nm0846894', 'nm3002376')]

def generate_candidate(x):
  candidates = []
  for a, elemA in enumerate(x):
    for b, elemB in enumerate(x):
      if a < b:
        candidates.append((elemA, elemB))
  return candidates

data.map(lambda x: x[1]).flatMap(lambda x: generate_candidate(x)) \
    .filter(lambda x: x in copia) \
    .map(lambda x: (x,1)) \
    .reduceByKey(lambda a,b: a+b) \
    .filter(lambda x: x[1] >=1) \
    .take(3)
          

# Apriori with MAP-REDUCE

Follow an implementatio of the Apriori algorithm using a map-reduce approach

In [67]:
def apriorihmap(data, support_threshold):
    """ 
    data: Pyspark.rdd 
      [
        [tconst, [nconst,]],
      ]
    """
    nconst_rdd = data.map(lambda x: x[1])

    frequent_items_rdd = nconst_rdd.flatMap(lambda x: x) \
          .map(lambda elem: (elem,1)) \
          .reduceByKey(lambda a,b: a+b) \
          .filter(lambda x: x[1] >= support_threshold)

    #print(f"found {frequent_items_rdd.count()} frequent singletons")
    frequent_singletons_bv = spark.sparkContext.broadcast({k[0]:True for k in frequent_items_rdd.collect()})

    def generate_candidate(x):
      candidates = []
      for a, elemA in enumerate(x):
        for b, elemB in enumerate(x):
          if a < b:
            # the tuple may be generated backwards, sort to get rid of the problem
            candidate_tuple = tuple(sorted((elemA, elemB)))
            candidates.append(candidate_tuple)
      return candidates
    
    frequent_couples_rdd = data.map(lambda x: x[1]) \
          .filter(lambda x: [elem for elem in x if frequent_singletons_bv.value.get(elem, False)])\
          .flatMap(lambda x: generate_candidate(x)) \
          .map(lambda x: (x,1)) \
          .reduceByKey(lambda a,b: a+b) \
          .filter(lambda x: x[1] >=support_threshold)

    return frequent_items_rdd.union(frequent_couples_rdd)

In [66]:
rules = apriorihmap(data, 60)
rules.count()

2209

In [68]:
rules = apriorihmap(minid, 5).collect()

In [69]:
rules

[('nm0165691', 5),
 ('nm0024706', 8),
 ('nm0330352', 5),
 ('nm0068213', 5),
 ('nm0926280', 12),
 ('nm0448682', 7),
 ('nm0817956', 5),
 ('nm0292810', 5),
 ('nm0624714', 5),
 ('nm0098376', 6),
 ('nm0292407', 8),
 ('nm0681933', 6),
 ('nm0731623', 6),
 ('nm0696049', 5),
 ('nm0516974', 6),
 ('nm0392059', 9),
 ('nm0148546', 5),
 ('nm0314700', 5),
 ('nm0829171', 7),
 ('nm0060238', 6),
 ('nm0624735', 5),
 ('nm0676473', 6),
 ('nm0163255', 5),
 ('nm0057585', 5),
 ('nm0728230', 5),
 ('nm0613115', 5),
 ('nm0386893', 5),
 (('nm0292407', 'nm0926280'), 5),
 (('nm0330352', 'nm0829171'), 5),
 (('nm0024706', 'nm0613115'), 5)]

In [17]:
len(rules)

28

In [ ]:
rules.take(5)

[('nm0003425', 62),
 ('nm0282348', 83),
 ('nm0723801', 126),
 ('nm0417837', 72),
 ('nm0165134', 77)]

# Apriori classic

In [53]:
from itertools import tee

def apriori(partitionData, support_threshold):
  singleton_counter = []
  lookup_index_table = {}
  reverse_lookup_index_table = {}
  
  d1, d2 = tee(partitionData, 2)

  # count singletons
  for bucket in d1:
    for item in bucket[1]:
      if item not in lookup_index_table:
        # The newly discovered element is appended on the tail of the array counter
        lookup_index_table[item] = len(singleton_counter)
        reverse_lookup_index_table[len(singleton_counter)] = item
        singleton_counter.append(0)
      idx = lookup_index_table[item]
      singleton_counter[idx] += 1

  frequent_items_table = [index for index,count in enumerate(singleton_counter) if count >= support_threshold]
  frequent_singleton = [(reverse_lookup_index_table[item], singleton_counter[item]) for item in frequent_items_table]
  # count pairs
  pair_counter = {}
  
  for bucket in d2:
      frequent_items_of_bucket = [lookup_index_table[item] for item in bucket[1] 
                        if lookup_index_table[item] in frequent_items_table]
      
      for x in frequent_items_of_bucket:
          for y in frequent_items_of_bucket:
              if x<y:
                  pair_counter[(x,y)] = pair_counter.get((x,y), 0) + 1

  # tuple(sorted(couple)) is done because may a couple be generated backward in a partition
  frequent_couples = [(tuple(sorted((reverse_lookup_index_table[couple[0]], reverse_lookup_index_table[couple[1]]))), count) for couple ,count 
                      in pair_counter.items() if count >= support_threshold]
  
  return iter(frequent_singleton + frequent_couples)

In [70]:
apriori_res = list(apriori(minid.collect(), 5))

In [71]:
apriori_res

[('nm0098376', 6),
 ('nm0292407', 8),
 ('nm0926280', 12),
 ('nm0448682', 7),
 ('nm0817956', 5),
 ('nm0165691', 5),
 ('nm0024706', 8),
 ('nm0829171', 7),
 ('nm0330352', 5),
 ('nm0681933', 6),
 ('nm0060238', 6),
 ('nm0731623', 6),
 ('nm0696049', 5),
 ('nm0624735', 5),
 ('nm0676473', 6),
 ('nm0516974', 6),
 ('nm0163255', 5),
 ('nm0057585', 5),
 ('nm0728230', 5),
 ('nm0392059', 9),
 ('nm0292810', 5),
 ('nm0613115', 5),
 ('nm0068213', 5),
 ('nm0386893', 5),
 ('nm0148546', 5),
 ('nm0624714', 5),
 ('nm0314700', 5),
 (('nm0292407', 'nm0926280'), 5),
 (('nm0330352', 'nm0829171'), 5),
 (('nm0024706', 'nm0613115'), 5)]

In [ ]:
len(apriori_res)

30

# SON

In [8]:
# empirical sweet-spot for the number of partitions (assuming every executor has 4 cores ...)
num_partitions = spark.sparkContext._jsc.sc().getExecutorMemoryStatus().size() * 4
num_partitions

4

In [9]:
d = data.repartition(num_partitions)

In [10]:
minid = d.take(500)
minid = spark.sparkContext.parallelize(minid)
minid = minid.repartition(num_partitions)
minid.take(5)

[Row(tconst='tt0001692', nconsts=['nm0003425', 'nm0282348', 'nm0016799', 'nm0643138', 'nm1270556']),
 Row(tconst='tt0001706', nconsts=['nm0001908', 'nm0138771', 'nm0865178', 'nm0693055', 'nm0639631', 'nm0533615']),
 Row(tconst='tt0001711', nconsts=['nm0009872', 'nm0002161', 'nm0166433', 'nm0179557', 'nm0723801', 'nm1356916']),
 Row(tconst='tt0001712', nconsts=['nm0054884']),
 Row(tconst='tt0001731', nconsts=['nm0198972', 'nm0519315', 'nm0528022'])]

In [40]:
def count_in_partition(data, frequent):
  # prepare data for processing
  frequent = frequent.value   # extract broadcasted values
  data = list(data)           # cast to list to iterate more than one time

  # check foreach frequent itemset
  for frequent_item in frequent:
    # trick to cast single element to list → not remove duplicate using set()
    if type(frequent_item) is not tuple:
      to_check = [frequent_item]
    else:
      to_check = frequent_item
      
    c = 0     # counter
    # and foreach row of the dataset
    for itemset in data:
      # check if the frequent itemset is subset of the items of the row
      if set(to_check).issubset(itemset[1]):
        c += 1
    yield (frequent_item, c)

In [47]:
def son_m_r(data, support):
  reduced_support = support//data.getNumPartitions()
  first_map = data.mapPartitions(lambda partition: apriori(partition, reduced_support)).map(lambda x: (x[0], 1))
  first_reduce = first_map.reduceByKey(lambda a,b: a+b)       # possible to remove a+b ?????????????????

  # extract the frequent items and broadcast them to worker nodes
  frequent_items = [x[0] for x in first_reduce.collect()]
  frequent_items = spark.sparkContext.broadcast(frequent_items)

  second_map = data.mapPartitions(lambda partition: count_in_partition(partition, frequent_items))
  second_reduce = second_map.reduceByKey(lambda a,b: a+b).filter(lambda x: x[1] >= support)
  return second_reduce

In [48]:
a = son_m_r(minid,5).collect()
a

[('nm0165691', 5),
 ('nm0024706', 8),
 ('nm0330352', 5),
 ('nm0068213', 5),
 ('nm0926280', 12),
 ('nm0448682', 7),
 ('nm0817956', 5),
 ('nm0292810', 5),
 ('nm0624714', 5),
 ('nm0098376', 6),
 ('nm0292407', 8),
 ('nm0681933', 6),
 ('nm0731623', 6),
 ('nm0696049', 5),
 ('nm0516974', 6),
 ('nm0392059', 9),
 ('nm0148546', 5),
 ('nm0314700', 5),
 ('nm0829171', 7),
 ('nm0060238', 6),
 ('nm0624735', 5),
 ('nm0676473', 6),
 ('nm0163255', 5),
 ('nm0057585', 5),
 ('nm0728230', 5),
 (('nm0292407', 'nm0926280'), 5),
 (('nm0330352', 'nm0829171'), 5),
 ('nm0613115', 5),
 ('nm0386893', 5),
 (('nm0024706', 'nm0613115'), 5)]

In [43]:
len(a)

30

# Demo FP Growht

In [23]:
basket_data = minid.toDF()
basket_data.show(10, False)

+---------+------------------------------------------------------------------+
|tconst   |nconsts                                                           |
+---------+------------------------------------------------------------------+
|tt0001692|[nm0003425, nm0282348, nm0016799, nm0643138, nm1270556]           |
|tt0001706|[nm0001908, nm0138771, nm0865178, nm0693055, nm0639631, nm0533615]|
|tt0001711|[nm0009872, nm0002161, nm0166433, nm0179557, nm0723801, nm1356916]|
|tt0001712|[nm0054884]                                                       |
|tt0001731|[nm0198972, nm0519315, nm0528022]                                 |
|tt0001756|[nm0034454, nm0034548, nm0679170, nm0688764]                      |
|tt0001790|[nm0470307, nm0959921, nm0893346, nm0592965]                      |
|tt0001812|[nm0207207, nm0294276]                                            |
|tt0001826|[nm0511729, nm0002001]                                            |
|tt0001889|[nm0143865, nm0159725, nm0483943, nm05390

In [25]:
from pyspark.ml.fpm import FPGrowth
fpGrowth = FPGrowth(itemsCol="nconsts", minSupport=0.01, minConfidence=0.01)
model = fpGrowth.fit(basket_data)

In [26]:
# Display frequent itemsets.
model.freqItemsets.show()
items = model.freqItemsets

+--------------------+----+
|               items|freq|
+--------------------+----+
|         [nm0516974]|   6|
|         [nm0613115]|   5|
|[nm0613115, nm002...|   5|
|         [nm0448682]|   7|
|         [nm0817956]|   5|
|         [nm0330352]|   5|
|[nm0330352, nm082...|   5|
|         [nm0068213]|   5|
|         [nm0926280]|  12|
|         [nm0624735]|   5|
|         [nm0829171]|   7|
|         [nm0057585]|   5|
|         [nm0696049]|   5|
|         [nm0392059]|   9|
|         [nm0060238]|   6|
|         [nm0148546]|   5|
|         [nm0624714]|   5|
|         [nm0024706]|   8|
|         [nm0163255]|   5|
|         [nm0165691]|   5|
+--------------------+----+
only showing top 20 rows



In [31]:
items.collect()

[Row(items=['nm0516974'], freq=6),
 Row(items=['nm0613115'], freq=5),
 Row(items=['nm0613115', 'nm0024706'], freq=5),
 Row(items=['nm0448682'], freq=7),
 Row(items=['nm0817956'], freq=5),
 Row(items=['nm0330352'], freq=5),
 Row(items=['nm0330352', 'nm0829171'], freq=5),
 Row(items=['nm0068213'], freq=5),
 Row(items=['nm0926280'], freq=12),
 Row(items=['nm0624735'], freq=5),
 Row(items=['nm0829171'], freq=7),
 Row(items=['nm0057585'], freq=5),
 Row(items=['nm0696049'], freq=5),
 Row(items=['nm0392059'], freq=9),
 Row(items=['nm0060238'], freq=6),
 Row(items=['nm0148546'], freq=5),
 Row(items=['nm0624714'], freq=5),
 Row(items=['nm0024706'], freq=8),
 Row(items=['nm0163255'], freq=5),
 Row(items=['nm0165691'], freq=5),
 Row(items=['nm0731623'], freq=6),
 Row(items=['nm0314700'], freq=5),
 Row(items=['nm0098376'], freq=6),
 Row(items=['nm0728230'], freq=5),
 Row(items=['nm0681933'], freq=6),
 Row(items=['nm0676473'], freq=6),
 Row(items=['nm0386893'], freq=5),
 Row(items=['nm0292407'], fr

In [ ]:
# Display generated association rules.
model.associationRules.show()
rules = model.associationRules

In [ ]:
# transform examines the input items against all the association rules and summarize the consequents as prediction
model.transform(basket_data).show()
transformed = model.transform(basket_data)

# Demo Antonio

In [ ]:
import pandas as pd

Lets try to load some data in a Pandas Dataframe

In [ ]:
actors_cols = {
    "original": [
        "nconst",  # actor unique id
        "knownForTitles"  # move he/she is in
    ],
    "renamed": ["actorId", "titles"]
}

actors_df = pd.read_csv(
    "name.basics.tsv.gz",
    compression="gzip",
    sep='\t',
    usecols=actors_cols["original"]
)

# clean and pre-process
actors_df.columns = actors_cols["renamed"]
actors_df.drop(actors_df[actors_df.titles == "\\N"].index, inplace=True)
actors_df.titles = actors_df.titles.apply(lambda x: x.split(","))

In [ ]:
actors_df

In [ ]:
def apriori(transactions, support_threshold):
    singleton_counter = []
    lookup_index_table = {}
    reverse_lookup_index_table = {}

    # count singletons
    for bucket in transactions:
        for elem in bucket:
            if elem not in lookup_index_table:
                # The newly discovered element is appended on the tail of the array counter
                lookup_index_table[elem] = len(singleton_counter)
                reverse_lookup_index_table[len(singleton_counter)] = elem
                singleton_counter.append(0)

            idx = lookup_index_table[elem]
            singleton_counter[idx] += 1

    frequent_items_table = [i for i,v in enumerate(singleton_counter) if v > support_threshold]

    # count pairs
    pair_counter = {}
    for bucket in transactions:
        frequent_items = [lookup_index_table[item] for item in bucket 
                          if lookup_index_table[item] in frequent_items_table]

        for x in frequent_items:
            for y in frequent_items:
                if x<y:
                    pair_counter[(x,y)] = pair_counter.get((x,y), 0) +1 

    return [list(map(lambda x: reverse_lookup_index_table[x], i)) for i,c in pair_counter.items() 
            if c > support_threshold] 

In [ ]:
apriori(data, 20)

TypeError: ignored

In [ ]:
# test
rules = apriori(actors_df.titles, 300)

movies_df = pd.read_csv("title.basics.tsv.gz", compression='gzip', sep='\t')
from IPython.display import display

for x,y in rules:
    display(movies_df.loc[((movies_df.tconst == x) | (movies_df.tconst == y))])